Return to Live Ocean Files and see if we can do better

In [46]:
import cmocean.cm as cm
import datetime
import importlib
import matplotlib.pyplot as plt
import mpl_toolkits.basemap as Basemap
import netCDF4 as nc
import numpy as np
import os
import scipy.interpolate as sinter
import xarray as xr

from salishsea_tools import LiveOcean_grid as grid
from salishsea_tools import gsw_calls
from salishsea_tools import LiveOcean_BCs as nancy

%matplotlib inline

In [47]:
importlib.reload(nancy)

<module 'salishsea_tools.LiveOcean_BCs' from '/ocean/sallen/allen/research/Meopar/Tools/SalishSeaTools/salishsea_tools/LiveOcean_BCs.py'>

# Define some stuff

In [48]:
# Create metadeta for temperature and salinity (Live Ocean variables, NEMO grid)
var_meta = {'vosaline': {'grid': 'SalishSea2',
                             'long_name': 'Practical Salinity',
                             'units': 'psu'},
                'votemper': {'grid': 'SalishSea2',
                             'long_name': 'Potential Temperature',
                             'units': 'deg C'},
                'NO3':      {'grid': 'SalishSea2',
                             'long_name': 'Nitrate',
                             'units': 'muM'},
                'Si':       {'grid': 'SalishSea2',
                             'long_name': 'Nitrate',
                             'units': 'muM'},
                }

# Mapping from LiveOcean TS names to NEMO TS names
LO_to_NEMO_var_map = {'salt': 'vosaline',
                          'temp': 'votemper',
                          'NO3': 'NO3',
                          'Si': 'Si'}


# Load BC information and Specify the New Grid
 

In [49]:
depBC, lonBC, latBC, shape = nancy.load_SalishSea_boundary_grid(imin=376-1, imax=470, rim=10, 
                                                          meshfilename='/home/sallen/MEOPAR/grid/mesh_mask201702.nc')

# Get the Live Ocean File

In [50]:
date = '2017-12-30'
d = nancy.load_LiveOcean(date)

# Process Live Ocean File

## Depth Interpolation

In [51]:
interps = nancy.interpolate_to_NEMO_depths(d, depBC)

## Convert to TEOS-10

In [52]:
var_meta, interps['salt'], interps['temp'] = \
            nancy._convert_TS_to_TEOS10(
                var_meta, interps['salt'], interps['temp'])

## Remove South of Tatoosh

In [53]:
interps = nancy.remove_south_of_Tatoosh(interps)

## Fill

In [54]:
interps = nancy.fill_box(interps)

## Convect

In [55]:
sigma = gsw_calls.generic_gsw_caller('gsw_sigma0.m', [interps['salt'][:], interps['temp'][:] ])

In [56]:
sigma, interps = nancy.convect(sigma, interps)

## Extend to Depth

In [57]:
interps = nancy.extend_to_depth(interps)

## Interpolate Horizontally onto NEMO grid

In [58]:
interpl = nancy.interpolate_to_NEMO_lateral(interps, d, lonBC, latBC, shape)

## Convect Again

In [59]:
sigmal = gsw_calls.generic_gsw_caller('gsw_sigma0.m', [interpl['salt'][:], interpl['temp'][:] ])

In [60]:
print(np.nanmax(sigmal[:-1, :, :] - sigmal[1:, :, :]))
sigmal, interpl = nancy.convect(sigmal, interpl)
print(np.nanmax(sigmal[:-1, :, :] - sigmal[1:, :, :]))

0.001
0.0


## Reorder Array

In [61]:
for var in interpl.keys():
    interpl[var] = np.swapaxes(interpl[var], 1, 2)
    interpl[var] = interpl[var].reshape(1, interpl[var].shape[0], 1, interpl[var].shape[2]*interpl[var].shape[1])


## Calculate Quantities LiveOcean doesn't Track like SI

In [62]:
interpl['Si'] = nancy.calculate_Si_from_NO3(interpl['NO3'])

# Write Out File

In [63]:
ts = d.ocean_time.data
ds = nancy.prepare_dataset(interpl, var_meta, LO_to_NEMO_var_map, depBC, ts)

In [64]:
ds

<xarray.Dataset>
Dimensions:       (deptht: 40, time_counter: 1, xbT: 950, yb: 1)
Coordinates:
  * yb            (yb) int64 1
  * deptht        (deptht) float64 0.5 1.5 2.5 3.5 4.5 5.5 6.5 7.501 8.501 ...
  * time_counter  (time_counter) datetime64[ns] 2017-12-30T12:00:00
  * xbT           (xbT) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
Data variables:
    vosaline      (time_counter, deptht, yb, xbT) float64 30.57 30.57 30.57 ...
    votemper      (time_counter, deptht, yb, xbT) float64 7.848 7.848 7.849 ...
    NO3           (time_counter, deptht, yb, xbT) float64 9.85 9.85 9.852 ...
    Si            (time_counter, deptht, yb, xbT) float64 11.6 11.6 11.6 ...
Attributes:
    institution:           UBC EOAS
    source:                http://nbviewer.jupyter.org/urls/bitbucket.org/sal...
    creator_name:          Salish Sea MEOPAR Project Contributors
    summary:               Temperature, Salinity and Nitrate from the Live Oc...
    acknowledgements:      Live Ocean h

In [65]:
basename = 'single_LO'
sdt = datetime.datetime.strptime(date, '%Y-%m-%d')
filename = '{}_{}.nc'.format(
                basename, sdt.strftime('y%Ym%md%d'))
print(filename)

single_LO_y2017m12d30.nc


In [66]:
encoding = {
    var: {
       'zlib': True
    }
    for var in ds.data_vars
}
encoding['time_counter'] = {'units': 'minutes since 1970-01-01 00:00'}

In [67]:
ds.to_netcdf(path='stuffit.nc', unlimited_dims=('time_counter'),
             encoding=encoding, 
             )       

In [68]:
os.rename('stuffit.nc', filename)